In [1]:
!pip install pyspark

1.b.Create a simple program that create a list of 10 random elements, put it on a rdd and divide the even digits by two while putting odd digits to the square.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

# Create a list of 10 random elements
import random
random_list = [random.randint(1, 100) for _ in range(10)]

# Parallelize the list into an RDD
rdd = spark.sparkContext.parallelize(random_list)

# Function to process elements based on condition
def process_element(num):
    if num % 2 == 0:  # Check if the number is even
        return num / 2
    else:
        return num ** 2

# Apply the function to each element of the RDD
processed_rdd = rdd.map(process_element)

# Collect the results and print them
result = processed_rdd.collect()
print("Original List:", random_list)
print("Processed List:", result)

Original List: [71, 22, 7, 55, 62, 6, 54, 11, 59, 75]
Processed List: [5041, 11.0, 49, 3025, 31.0, 3.0, 27.0, 121, 3481, 5625]


1.c.write a program to load the json file movies-2020s (do not forget the option(’multilines’, True)). Give the live of Horror movies of 2011


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, array_contains

# Initialize Spark session
spark = SparkSession.builder.appName("Movies Analysis").getOrCreate()

# Load the JSON file
movies_df = spark.read.option("multiline", True).json("movies-2020s.json")

# Filter for Horror movies from 2021
horror_movies_2011_df = movies_df.filter(
    array_contains(col("genres"), "Horror") & (col("year") == 2021)
)

# Show the result for Horror movies of 2021
print("Horror Movies of 2011:")
horror_movies_2011_df.show(truncate=False)

Horror Movies of 2011:
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# Load the JSON file
movies_df = spark.read.option("multiline", True).json("movies-2020s.json")

from pyspark.sql.functions import array_contains

# Filter Horror movies
horror_movies_df = movies_df.filter(array_contains(col("genres"), "Horror"))

# Show the result
horror_movies_df.show(truncate=False)


+-----------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------+----------------------------------+-----------------------------------------------------------------------------------------------------------+----------------+---------------+----------------------------------+----+
|cast          

1.dWhich movies has the longest abstract?

In [5]:
# Calculate the lengths of the 'extract' field (assuming this is the abstract)
movies_df = movies_df.withColumn("extract_length", length(col("extract")))

# Find the movie with the longest 'extract'
longest_extract_movie = movies_df.orderBy(col("extract_length").desc()).select("title", "extract_length").first()

# Print the result for the movie with the longest 'extract'
print("\nMovie with the Longest Extract:")
print("Title:", longest_extract_movie["title"])
print("Extract Length:", longest_extract_movie["extract_length"])


Movie with the Longest Extract:
Title: Puss in Boots: The Last Wish
Extract Length: 1389



2.a) Create a function to genereate a random point (use random.random() ) and compute if this point is in the unit circle. Your function must take a number as argument (even if it does not use it)

In [6]:
from pyspark import SparkContext
import random

def is_point_in_unit_circle(point):
    x, y = point
    return x**2 + y**2 <= 1

2.b) Test your function on some iteration

In [7]:
for _ in range(5):
        print(is_point_in_unit_circle((random.random(), random.random())))

False
False
False
True
True


2.c) Now, let’s use the power of Spark to compute the approximation of pi: we will create a RDD with N values inside. We can now filter this RDD. filter is a method that take a boolean function as argument. It then applu this function to every element of the RDD and only keep the ones that returned true (hence the need of a value as argument for our function).
d) We now only need to count() the number of entry and multiply it by N4

In [8]:
def approximate_pi_spark(sc, N):
    # Create an RDD with N random points
    points_rdd = sc.parallelize([(random.random(), random.random()) for _ in range(N)])

    # Use filter to keep only points inside the unit circle
    inside_circle_rdd = points_rdd.filter(is_point_in_unit_circle)

    # Count the number of points inside the unit circle
    inside_circle_count = inside_circle_rdd.count()

    # Compute the approximation of pi
    pi_approximation = (inside_circle_count / N) * 4

    return pi_approximation

2.e)Try this method with N=1000, 50000, 800000.

In [9]:
from pyspark import SparkContext

def main():
    # Check if a SparkContext already exists
    sc = SparkContext.getOrCreate()

    # Use Spark to approximate pi with different values of N
    N_values = [1000, 50000, 800000]
    for N in N_values:
        pi_approximation = approximate_pi_spark(sc, N)
        print(f"Approximation of pi with N={N}: {pi_approximation}")


if __name__ == "__main__":
    main()


Approximation of pi with N=1000: 3.144
Approximation of pi with N=50000: 3.1352
Approximation of pi with N=800000: 3.144155


2.f) if we ofter repeat a method with the exact same parameters, we like to embedded in another method. This will be the case as we want to write every answer in the same file/folder. This method must check if the object is an RDD, a DataFrame or another class to determine which write methode to use.

In [10]:
def write_spark_object(spark_object, file_path):
    if isinstance(spark_object, RDD):
        spark_object.saveAsTextFile(file_path)
    elif isinstance(spark_object, DataFrame):
        spark_object.write.csv(file_path)
    else:
        with open(file_path, 'w') as file:
            file.write(str(spark_object))

3.a) Read the movie.csv file with the read command and the header option. How many lines are in the dataframe? You can access the columns and their types with df.columns and df.dtypes

In [11]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Movies Analysis").getOrCreate()

# Read the 'movies.csv' file
df = spark.read.option("header", True).csv("movies.csv")

# Count the number of lines
line_count = df.count()
print("Number of lines in the DataFrame:", line_count)

# Access columns and their types
print("Columns:", df.columns)
print("Data Types:", df.dtypes)

Number of lines in the DataFrame: 45572
Columns: ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']
Data Types: [('adult', 'string'), ('belongs_to_collection', 'string'), ('budget', 'string'), ('genres', 'string'), ('homepage', 'string'), ('id', 'string'), ('imdb_id', 'string'), ('original_language', 'string'), ('original_title', 'string'), ('overview', 'string'), ('popularity', 'string'), ('poster_path', 'string'), ('production_companies', 'string'), ('production_countries', 'string'), ('release_date', 'string'), ('revenue', 'string'), ('runtime', 'string'), ('spoken_languages', 'string'), ('status', 'string'), ('tagline', 'string'), ('title', 'string'), ('video', 'string'), ('vote_average', 'string'),

3.b)We can access the column of the df with df.col_name. The method filter or where behave like the previous filter. Use this to get the number of movies from 2012


In [12]:
from pyspark.sql.functions import year

movies_2012_count = df.filter(year(df.release_date) == 2012).count()
print("Number of movies from 2012:", movies_2012_count)


Number of movies from 2012: 1599


3.c)Use the select to keep the title, budget and revenue from movie that were profitable


In [13]:
profitable_movies = df.filter(df.revenue > df.budget).select("title", "budget", "revenue")
profitable_movies.show()

+--------------------+--------+--------------------+
|               title|  budget|             revenue|
+--------------------+--------+--------------------+
|           Toy Story|30000000|           373554033|
|[{'iso_639_1': 'e...|16000000|[{'name': 'Twenti...|
|Father of the Bri...|       0|            76578911|
|        Sudden Death|35000000|            64350171|
|               Balto|       0|            11348324|
|          Four Rooms| 4000000|             4300000|
|   Leaving Las Vegas| 3600000|            49800000|
|        Now and Then|12000000|            27400000|
|     Dangerous Minds|       0|           180000000|
|    Dead Man Walking|11000000|            39363635|
|Cry, the Beloved ...|       0|              676525|
|[{'iso_639_1': 'e...|10000000|[{'name': 'Carava...|
|          To Die For|20000000|            21284514|
|How To Make An Am...|10000000|            23574130|
|[{'iso_639_1': 'e...|33000000|[{'name': 'New Li...|
|    Mighty Aphrodite|15000000|             67

3.d)The colRegex method is useful to put a regex. Use it to fine every movies with a number in it’s title

In [14]:
movies_with_number = df.filter(df.title.rlike("\d"))
movies_with_number.show()

+-----+---------------------+--------+--------------------+--------------------+------+---------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|adult|belongs_to_collection|  budget|              genres|            homepage|    id|  imdb_id|original_language|      original_title|            overview|          popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|              status|             tagline|               title|               video|        vote_average|          vote_count|
+-----+---------------------+--------+--------------------+--------------------+------+---------

3.e)The dataframe can run the join command, to perform join on table when possinle. We can now use the rating table to get a dataframe with the rating, budget, revenue, date (year), orignal language.

In [15]:
# Read the ratings CSV file
ratings_df = spark.read.option("header", True).csv("ratings.csv")

# Perform the join
joined_df = df.join(ratings_df, df.id == ratings_df.movieId).select(df.title, df.budget, df.revenue, df.release_date, df.original_language, ratings_df.rating)

# Show the joined DataFrame
joined_df.show()

+----------------+--------+---------+------------+-----------------+------+
|           title|  budget|  revenue|release_date|original_language|rating|
+----------------+--------+---------+------------+-----------------+------+
|            Heat|60000000|187436818|  1995-12-15|               en|   4.0|
|            Heat|60000000|187436818|  1995-12-15|               en|   4.0|
|            Heat|60000000|187436818|  1995-12-15|               en|   4.0|
|            Heat|60000000|187436818|  1995-12-15|               en|   4.0|
|            Heat|60000000|187436818|  1995-12-15|               en|   4.0|
|       GoldenEye|58000000|352194034|  1995-11-16|               en|   2.5|
|Cutthroat Island|98000000| 10017322|  1995-12-22|               en|   4.5|
|Cutthroat Island|98000000| 10017322|  1995-12-22|               en|   4.5|
|Cutthroat Island|98000000| 10017322|  1995-12-22|               en|   5.0|
|Cutthroat Island|98000000| 10017322|  1995-12-22|               en|   3.0|
|Cutthroat I

3.f)Check the Schema of the last dataframe obtained. If needed cast the columns into float (use cast(’float’))

In [16]:
# Check the schema
joined_df.printSchema()

# Cast columns to float if needed
joined_df = joined_df.withColumn("budget", joined_df.budget.cast('float')).withColumn("revenue", joined_df.revenue.cast('float'))

# Check the schema again after casting
joined_df.printSchema()

root
 |-- title: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- rating: string (nullable = true)

root
 |-- title: string (nullable = true)
 |-- budget: float (nullable = true)
 |-- revenue: float (nullable = true)
 |-- release_date: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- rating: string (nullable = true)



4.a) First step, as always, load the file into spartk. How many rows are in the file? Make sure that you have a DataFrame object

In [17]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Titanic Data Analysis").getOrCreate()

# Load the Titanic dataset
titanic_df = spark.read.option("header", True).csv("train.csv")

# Count the number of rows
row_count = titanic_df.count()
print("Number of rows in the Titanic dataset:", row_count)

Number of rows in the Titanic dataset: 891


4.b) The command df.toPandas() shoudl make for a better vizualisation. However it will only work with small dataset as the data must be in memory

In [18]:
# Convert to Pandas DataFrame for visualization
titanic_pd_df = titanic_df.toPandas()
titanic_pd_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S


4.c) Convert the columns that should be float (show the first values to see when you do need a float)

In [19]:
from pyspark.sql.functions import col

# Convert 'Age' and 'Fare' to float
titanic_df = titanic_df.withColumn("Age", col("Age").cast('float'))
titanic_df = titanic_df.withColumn("Fare", col("Fare").cast('float'))

# Convert 'Survived', 'Pclass', 'SibSp', and 'Parch' to integer
titanic_df = titanic_df.withColumn("Survived", col("Survived").cast('integer'))
titanic_df = titanic_df.withColumn("Pclass", col("Pclass").cast('integer'))
titanic_df = titanic_df.withColumn("SibSp", col("SibSp").cast('integer'))
titanic_df = titanic_df.withColumn("Parch", col("Parch").cast('integer'))

# Show the first few values to verify the changes
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

4.d) Use the describe() method to have a good view of the dataset

In [20]:
# Describe the dataset
titanic_description = titanic_df.describe().toPandas()
titanic_description


,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764704046,0.5230078563411896,0.38159371492704824,260318.54916792738,32.20420804114722,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332370992,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342916316158,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,99,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


Question 5
a) The call count(isnull(c), c)) will act as an if to catch None values. We will call this method for each column. We now only need to count the values.
b) You should only having 2 Embarked and 177 Age missing
c) Use show and some queries to understand which character is used to represent missing values. Replace it with None
d) The dropna(how=’any) call will remove every nulls values
Almost done! We need to make sure that we work with numerical data:
• StringIndexer(inputCol=,outputCol=,handleInvalid=’keep’).fit(dataset).transform(dataset)willreturnadataset where the specified column has been added with a number. Use this approch to only have numerical values
• You can now drop the columns with no numerical attributes

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Assuming 'dataset' is your PySpark DataFrame
for column in titanic_df.columns:
    null_count = titanic_df.filter(col(column).isNull()).count()
    print(f"Number of null values in column '{column}': {null_count}")


Number of null values in column 'PassengerId': 0
Number of null values in column 'Survived': 0
Number of null values in column 'Pclass': 0
Number of null values in column 'Name': 0
Number of null values in column 'Sex': 0
Number of null values in column 'Age': 177
Number of null values in column 'SibSp': 0
Number of null values in column 'Parch': 0
Number of null values in column 'Ticket': 0
Number of null values in column 'Fare': 0
Number of null values in column 'Cabin': 687
Number of null values in column 'Embarked': 2


In [22]:
embarked_missing_count = titanic_df.filter(col('Embarked').isNull()).count()
age_missing_count = titanic_df.filter(col('Age').isNull()).count()

print(f"Number of missing values in 'Embarked': {embarked_missing_count}")
print(f"Number of missing values in 'Age': {age_missing_count}")


Number of missing values in 'Embarked': 2
Number of missing values in 'Age': 177


In [23]:
from pyspark.sql.functions import col, when
# Identifying missing values in 'age' column and storing the missing values character
missing_values_char = titanic_df.select("age").distinct().filter(col("age").isNull()).head()[0]
missing_values_char = str(missing_values_char)

# Replacing missing values in 'age' column with 'None'
titanic_df = titanic_df.withColumn("age", when(col("age") == missing_values_char, None).otherwise(col("age")))

# Dropping rows with any null values from the DataFrame
titanic_df_no_nulls = titanic_df.dropna(how='any')
titanic_df = titanic_df.dropna(subset=["Age", "Embarked"])
# Displaying the modified DataFrame
titanic_df_no_nulls.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|     113803|    53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|      17463| 51.8625|        E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|     113783|   26.55|       C103|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|  male|34.0|  

In [24]:
from pyspark.ml.feature import StringIndexer

# Convert 'Sex' column to numerical
indexer_sex = StringIndexer(inputCol="Sex", outputCol="SexIndexed", handleInvalid='keep')
titanic_df = indexer_sex.fit(titanic_df).transform(titanic_df)

# Convert 'Embarked' column to numerical
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndexed", handleInvalid='keep')
titanic_df = indexer_embarked.fit(titanic_df).transform(titanic_df)


In [25]:
from pyspark.ml.feature import StringIndexer
# StringIndexer to convert string columns to numerical data
string_indexer = StringIndexer(inputCol="age", outputCol="age_numeric_new", handleInvalid="keep")
titanic_df = string_indexer.fit(titanic_df).transform(titanic_df)

# Drop columns with no numerical attributes
numeric_columns = [col for col, dtype in titanic_df.dtypes if dtype != "string"]
titanic_df = titanic_df[numeric_columns]
# Display the modified DataFrame with numerical data only
titanic_df.show()

+--------+------+----+-----+-----+-------+----------+---------------+---------------+
|Survived|Pclass| age|SibSp|Parch|   Fare|SexIndexed|EmbarkedIndexed|age_numeric_new|
+--------+------+----+-----+-----+-------+----------+---------------+---------------+
|       0|     3|22.0|    1|    0|   7.25|       0.0|            0.0|            1.0|
|       1|     1|38.0|    1|    0|71.2833|       1.0|            1.0|           26.0|
|       1|     3|26.0|    0|    0|  7.925|       1.0|            0.0|           10.0|
|       1|     1|35.0|    1|    0|   53.1|       1.0|            0.0|           13.0|
|       0|     3|35.0|    0|    0|   8.05|       0.0|            0.0|           13.0|
|       0|     1|54.0|    0|    0|51.8625|       0.0|            0.0|           32.0|
|       0|     3| 2.0|    3|    1| 21.075|       0.0|            0.0|           25.0|
|       1|     3|27.0|    0|    2|11.1333|       1.0|            0.0|           11.0|
|       1|     2|14.0|    1|    0|30.0708|       1.0| 

6
a) Create a list with the name of the 5 column that you will use for the model.
b) Import the VectorAssembler class from the pyspark.ml.feature lib
c) assembler=VectorAssembler(inputCols=required_features,outputCol=’features’)andthentransformthedataset and put them in transformed_data
d) We can now train a model: assembler = VectorAssembler(inputCols=required_features, outputCol=’features’)
e) As always, we need to split the dataset between training_data and test_data
f) Let’s import the RandomForstClassifierfrom pyspark.ml.classification import RandomForestClassifier
g) create a RandomForest with the following argument: labelCol=’Survived’, featuresCol=’features’, maxDepth=5 As you can see, we can only provide one column to the classifier
h) Actually train the model: call the fit method from the randomForest on training_data
i) The model onject has a method to turn a RDD (test_data) into a prediction with the transform method
j) Last step, we need an evaluator. imort the lib: from pyspark.ml.evaluation import MulticlassClassificationEval- uator
k) Create a MultiClassClassificationEvaluatore with labelCol=’Survived’, predictionCol=’prediction’, metricName=’accuracy
l) Finally, you can call the evaluate method and print the result! Try different values and split to see which one give you the better values!
m) Use the same approach to compute the correlation matrix of the last movies tables we obtained (method corr de Correlation, se trouvant dans le package pyspark.ml.stat.

In [29]:
required_features = ["Pclass", "age", "Fare", "Survived", "EmbarkedIndexed"]


In [30]:
from pyspark.ml.feature import VectorAssembler


In [31]:
assembler = VectorAssembler(inputCols=required_features, outputCol='features')
transformed_data = assembler.transform(titanic_df)

In [32]:
training_data, test_data = transformed_data.randomSplit([0.9, 0.09], seed=42)

In [33]:
from pyspark.ml.classification import RandomForestClassifier
random_forest = RandomForestClassifier(labelCol='Survived', featuresCol='features', maxDepth=5)
rf_model = random_forest.fit(training_data)
predictions = rf_model.transform(test_data)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [34]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation



# Converting 'rating' to float if it's numeric
joined_df = joined_df.withColumn("rating", col("rating").cast('float'))

# Select only the columns that are relevant and already in a numeric format
selected_columns = ['budget', 'revenue', 'rating']

# Remove rows with null values in the selected columns
cleaned_joined_df = joined_df.dropna(subset=selected_columns)

# Assemble the features into a vector
assembler = VectorAssembler(inputCols=selected_columns, outputCol="features")
joined_df_vector = assembler.transform(cleaned_joined_df)

# Compute the correlation matrix
correlation_matrix = Correlation.corr(joined_df_vector, "features").head()

# Print the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix[0])


Correlation Matrix:
DenseMatrix([[ 1.        ,  0.71188117, -0.00246142],
             [ 0.71188117,  1.        , -0.00184857],
             [-0.00246142, -0.00184857,  1.        ]])
